In [ ]:
import json
import logging
import os
from typing import Any

import nltk
import pandas as pd
from dotenv import load_dotenv
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Load environment variables
load_dotenv()

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ensure NLTK resources are downloaded once
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load dataset
DATASET_PATH = os.getenv("DATASET_PATH", "../data/dev.json")
df = pd.read_json(DATASET_PATH)



In [ ]:

# Schema extraction from CSV
def get_table_schema_from_csv(csv_path: str) -> dict:
    df_csv = pd.read_csv(csv_path, nrows=0)
    table_name = os.path.basename(csv_path).replace(".csv", "")
    columns = df_csv["column_name"]
    column_description = df_csv["column_description"] if "column_description" in df_csv.columns else [""]

    return {
        "table_name": table_name,
        "columns": columns,
        "table_description": column_description
    }


# Tokenization and lemmatization
def tokenize_and_lemmatize(question):
    tokens = word_tokenize(question)
    pos_tags = pos_tag(tokens)
    irrelevant_pos = {"DT", ".", ",", ":", "''", "``", "(", ")"}

    return [
        [token, lemmatizer.lemmatize(token.lower()), pos]
        for token, pos in pos_tags if pos not in irrelevant_pos
    ]


# Apply tokenization and lemmatization
df["tokens"] = df["question"].apply(tokenize_and_lemmatize)

# Prepare lemmas for TF-IDF
df["lemmas_str"] = df["tokens"].apply(lambda toks: " ".join(lemma for _, lemma, _ in toks))
df

In [ ]:

# Compute TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["lemmas_str"])
feature_names = vectorizer.get_feature_names_out()


# Attach TF-IDF scores to tokens
def attach_tfidf(tokens, doc_index):
    tfidf_scores = tfidf_matrix[doc_index].toarray().flatten()
    scores_dict = dict(zip(feature_names, tfidf_scores))

    return [
        [token, lemma, pos, scores_dict.get(lemma, 0.0)]
        for token, lemma, pos in tokens
    ]


df["tokens"] = df.apply(lambda row: attach_tfidf(row["tokens"], row.name), axis=1)

# Filter tokens based on TF-IDF threshold (70% of max)
df["filtered_tokens"] = df["tokens"].apply(
    lambda tokens: [
        [token, lemma, pos, float(score)]
        for token, lemma, pos, score in tokens
        if score >= 0.6 * max(s[3] for s in tokens)
    ]
)
df


In [ ]:

from llm.llm_request import call_llm_model


# Format prompt for LLM
def format_prompt(tokens: list, table_info: dict, question: str):
    token_texts = [token[0] for token in tokens if token[0]]
    token_texts = list(set(token_texts))

    tables_and_columns = []
    for table in table_info['table_name']:
        cols = table_info['columns'][table]
        column_entries = [
            f"- {col['column_name']}: {col['column_description']}" if col[
                'column_description'] else f"- {col['column_name']}"
            for col in cols
        ]
        tables_and_columns.append(f"TABLE: {table}\nCOLUMNS:\n" + "\n".join(column_entries))

    tables_and_columns_str = "\n\n".join(tables_and_columns)

    return f"""Given the following tables and columns information:
{tables_and_columns_str}

Relevant keywords extracted from the user's question: {', '.join(token_texts)}

User question: {question}

Map each relevant keyword to the most appropriate column and corresponding table from the information above. Provide your answer strictly as a JSON object. Do not include any explanations or text outside the JSON structure. Only include mappings you are confident about.

Example output:
{{
    "year": {{
        "column_name": "year",
        "table_name": "movies"
    }},
    "sales_amount": {{
        "column_name": "amount",
        "table_name": "sales"
    }}
}}
"""


# Extract only JSON object from LLM output
def extract_json_from_response(response_text):
    import json
    import logging

    stack = []
    start = None

    for i, char in enumerate(response_text):
        if char == '{':
            if not stack:
                start = i
            stack.append(char)
        elif char == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    json_candidate = response_text[start:i + 1]
                    try:
                        return json.loads(json_candidate)
                    except json.JSONDecodeError:
                        logging.warning("Found braces but content is not valid JSON.")
                        return {
                            "mapping": f"{response_text}"
                        }

    logging.warning("No valid JSON object found in the response.")
    return {
        "mapping": f"{response_text}"
    }


# Main function to get mappings using LLM
def get_token_column_mapping(row) -> Any:
    tables_path = os.path.join("../data/dev_databases", row['db_id'], "database_description")
    table_names = []
    columns_info = {}

    for file in os.listdir(tables_path):
        if file.endswith(".csv"):
            df_file = pd.read_csv(os.path.join(tables_path, file), encoding_errors='replace')
            table_name = file.replace(".csv", "")
            table_names.append(table_name)

            if "original_column_name" in df_file.columns:
                descriptions = df_file["original_column_name"].fillna("").astype(str).tolist()
            else:
                descriptions = [""] * len(df_file)

            columns = [
                {
                    'column_name': str(col_name),
                    'column_description': desc
                }
                for col_name, desc in zip(df_file["original_column_name"], descriptions)
            ]

            columns_info[table_name] = columns

    flattened_table_info = {
        'table_name': table_names,
        'columns': columns_info
    }

    prompt = format_prompt(row["filtered_tokens"], flattened_table_info, row["question"])
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

    try:
        res = call_llm_model({
            "prompt": prompt,
            "temperature": 0.1,
            "max_tokens": 512,
            "top_k": 2,
            "top_p": 0.9,
        }, model_id=model_id)

        parsed = extract_json_from_response(res)
        logging.info(f"Processed {row['question_id']} ({row.name + 1}/{len(df)})")
        return json.dumps(parsed)

    except Exception as e:
        logging.error(f"Failed to process row {row.name}: {e}")
        return json.dumps({})


# Apply the function to get mappings
df["token_column_mapping"] = df.apply(get_token_column_mapping, axis=1)


In [ ]:
# Save the enriched dataset
OUTPUT_PATH = os.getenv("OUTPUT_PATH", "../data/dev_enriched.json")

json_data = []

for index, row in df.iterrows():
    json_data.append({
        "question_id": row['question_id'],
        "db_id": row['db_id'],
        "question": row['question'],
        "evidence": row['evidence'],
        "SQL": row['SQL'],
        "difficulty": row['difficulty'],
        "filtered_tokens": row['filtered_tokens'],
        "token_column_mapping": json.loads(row['token_column_mapping'])
    })

with open(OUTPUT_PATH, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)